# Dynamical System Approximation

In this notebook we implement for experimental purposes the Salsa algorithm from Kraemer et. al. (Stable ALS Approximation in the TT-Format for Rank adaptive Tensor Completion.

We use a mixture from Xerus and Numpy to deal with Tensors.

Again, we try to learn  functional correlations based on given snapshots. The data is created through the following ODE:
\begin{align}
\frac{d^2}{dt^2} x_i = (x_{i+1} - 2x_i + x_{i-1}) + 0.7((x_{i+1} - x_i)^3 - (x_i-x_{i-1})^3)
\end{align}

In [ ]:
import numpy as np
import xerus

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import time as _time
from itertools import chain
import helpers as hp
import pandas as pd


## Salsa implementation
Now we can try to recover the exact solution with the SALS algorithm. 

### Intialization
We intialize Salsa with a random rank one Tensor and kmin additional ranks.

In [ ]:
def initialize(p,noo,kmin):
    dim = [p for i in range(0,noo)]
    dim.append(noo)
    ranks = [kmin + 1 for i in range(0,noo)]
    X = xerus.TTTensor.random(dim,ranks)
    X.move_core(0)
    X = 0.0*X
    omega = 0.5
    r1 = 1
    r2 = 1
    maxrank1 = []
    maxrank2 = []
    for i in range(len(ranks)):
        r1 = r1*dim[i]
        r2 = r2*dim[noo-i]
        maxrank1.append(r1)
        maxrank2.append(r2)
    maxrank2.reverse()
    maxranks =np.minimum(maxrank1,maxrank2)
    return X, maxranks


### Perform one SALSA sweep

In [ ]:
# build exact solution
def exact(noo,p,b):
    beta = 0.7
    dim = [p for i in range(0,noo)]
    dim.append(noo)
    Solution = xerus.TTTensor(dim)

    tmp = xerus.Tensor([1,4,4*noo])
    for eq in range(noo):
        tmp[0,0,4*eq] = 1
    tmp[0,0,0] = 0
    tmp[0,1,0] = -2 
    tmp[0,3,0] = -2*beta
    tmp[0,0,1] = 1
    tmp[0,2,1] = 3*beta
    tmp[0,1,2] = -3*beta
    tmp[0,0,3] = beta

    tmp[0,0,4] = 1
    tmp[0,1,5] = 1
    tmp[0,2,6] = 1
    tmp[0,3,7] = 1
    Solution.set_component(0,tmp)


    for comp in range(1,Solution.order()-1):
        tmp = xerus.Tensor([4*noo,4,4*noo])
        for eq in range(noo):
            tmp[4*eq,0,4*eq] = 1
        if (comp+1)*4 < 4*noo:
            tmp[4*(comp+1),0,4*(comp+1)] = 1
            tmp[4*(comp+1),1,4*(comp+1)+1] = 1
            tmp[4*(comp+1),2,4*(comp+1)+2] = 1
            tmp[4*(comp+1),3,4*(comp+1)+3] = 1

        tmp[4*comp,0,4*comp] = 0
        tmp[4*comp,1,4*comp] = -2 
        tmp[4*comp,3,4*comp] = -2*beta
        tmp[4*comp+1,0,4*comp] = 1       
        tmp[4*comp+1,2,4*comp] = 3*beta
        tmp[4*comp+2,1,4*comp] = -3*beta
        tmp[4*comp+3,0,4*comp] = beta
        tmp[4*comp,0,4*comp+1] = 1       
        tmp[4*comp,2,4*comp+1] = 3*beta
        tmp[4*comp,1,4*comp+2] = -3*beta
        tmp[4*comp,0,4*comp+3] = beta

        tmp[4*(comp-1),0,4*(comp-1)] = 1
        tmp[4*(comp-1)+1,1,4*(comp-1)] = 1
        tmp[4*(comp-1)+2,2,4*(comp-1)] = 1
        tmp[4*(comp-1)+3,3,4*(comp-1)] = 1

        Solution.set_component(comp,tmp)

    tmp = xerus.Tensor([4*noo,noo,1])
    for eq in range(noo):
        tmp[4*eq,eq,0] = 1
    Solution.set_component(Solution.order()-1,tmp)
    Solution.round(0.0)

    M = np.zeros([4,4])
    if b == 0:
        M[0,0] = 1   #1
        M[1,1] = 1   #1
        M[2,2] = 1.5 #1.5
        M[3,3] = 2.5 #2.5
        M[0,2] = -0.5 #-0.5
        M[1,3] = -1.5 #-1.5
    elif b == 1:
        M[0,0] = 1/np.sqrt(2)   #1
        M[1,1] = np.sqrt(3/8)    #1
        M[2,2] = np.sqrt(45/128) #1.5
        M[3,3] = 5*np.sqrt(7/302)  #2.5
        M[0,2] = -np.sqrt(5/128)  #-0.5
        M[1,3] = -9/2*np.sqrt(7/302) #-1.5
    elif basis == 2:
        M = np.identity(4)
    t = xerus.Tensor.from_ndarray(np.linalg.inv(M))
    a1,a2,a3,a4,b1,b2,b3,b4 = xerus.indices(8)
    for eq in range(noo):
        tmp = Solution.get_component(eq)
        tmp(a1,a2,a3) <<  tmp(a1,b2,a3)* t(a2,b2)
        Solution.set_component(eq,tmp)

    return Solution

    

In [ ]:
#Master iteration
data_noo_nos = [(8,2000)] #specify pairs to simulate for
runs = 10
max_iter = 100

tuples = []
for data in data_noo_nos:
    noo = data[0]
    nos = data[1] 
    for r in range(0,runs):
        tuples.append((noo,nos,r))

index = pd.MultiIndex.from_tuples(tuples, names=['d', 'm','runs'])           

df = pd.DataFrame(np.zeros([len(tuples),max_iter]), index=index)
print(len(index))
print(data_noo_nos)

output = 'data.csv'


In [ ]:
b = 0
eps = 1e-10
lam = 1
kmin = 2

psi = hp.basis(b)
p = len(psi)
for data in data_noo_nos:
    noo = data[0]
    nos = data[1]
    print( "(noo,nos) = (" + str(noo) +',' + str(nos) + ')' )

    Solution = exact(noo,p,b)
    print("Solution frob_norm: " +str(Solution.frob_norm()))
    for r in range(runs):
        [x, y] = hp.fermi_pasta_ulam(noo, nos)
        Alist = hp.build_data_tensor_list2(noo,x,nos,psi,p)
        print(Alist[0].dimensions[0])
        Y = xerus.Tensor.from_ndarray(y)
        X, maxranks = initialize(p,noo, kmin)
        errors = hp.run_salsa(X,noo,Alist,nos,Solution,Y,max_iter,maxranks, kmin)
        for i in range(len(errors)):
            df[i].loc[(noo,nos,r)] = errors[i]
        print("Run: " +str(r) + " finished result = " + str(errors))
        df.to_csv(output)